# Michelson tutorial
## Chapter seven

This new chapter will be the occasion to work a little more in-depth with two data types that are pretty important on a smart contract: strings and lists.

In most projects that you will work on, you will probably have to manipulate strings. Whether you want to save some data provided by your users or compare inputs, strings are an essential component of smart contracts. Unlike other smart contract languages whose developers have been asking for these features for years (for example Solidity), Michelson offers string manipulation functions out of the box. Although still limited compared to other lhigh level languages, these functions will still provide enough flexibility to get the job done.

Once you have strings in your smart contract, what about storing them somewhere? Lists could be the perfect place to put them! Michelson offers different possibilities in terms of storage for multiple pieces of data and lists are one of them. Before choosing a list for your needs, you must know more about what make them different and how they work. This will have no more secret for you at the end of this tutorial!

- string type
- _CONCAT_, _SLICE_, _SIZE_, _COMPARE_
- list type
- *CONS*, *NIL*, *IF_CONS*, *SIZE*

##### 1. Strings and string manipulations

We have already worked with strings in the previous chapters, so you should already be familiar with this type. Strings are a series of characters delimited by double-quotes and including only the characters present in the English alphabet (and the escape character in [certain circumstances](https://tezos.gitlab.io/whitedoc/michelson.html#constants)). Strings are comparable values that can be added to contract from different sources: they can come from the parameter, the storage or the **`PUSH`** instruction.

Once the string is present in the stack, you can work with it!  
First, you can check the length of the string with the **`SIZE`** opcode:

In [1]:
storage nat ;
parameter string ;
code {
    CAR ;
    SIZE ;
    NIL operation ;
    PAIR
};

RUN %default "tezos" 0;

storage nat;
parameter string;
code { CAR ; SIZE ; NIL operation ; PAIR };
RUN: use %default; drop all; push ('tezos', 0);
CAR: pop ('tezos', 0); push tezos;
SIZE: pop tezos; push 5;
NIL: push [];
PAIR: pop [], 5; push ([], 5);

value,type
5,nat


As expected, the length of the string *"Tezos"* is `5`. Note that the length is returned as a `nat` because negative length are not possible.  
Knowing the length of a string can be useful if you want to limit the size of the strings savec into your contract. Let's write an example that will reject string parameters if their length exceeds `5`:

In [18]:
storage string ;
parameter string ;
code {
    CAR ;
    DUP ;
    SIZE ;
    PUSH nat 5 ;
    IFCMPLT
        { FAIL }
        {
            NIL operation ;
            PAIR
        }
} ;

RUN %default "tezos" "" ;

storage string;
parameter string;
code { CAR ; DUP ; SIZE ; PUSH nat 5 ; { { COMPARE ; LT } ; IF { { UNIT ; FAILWITH } } { NIL operation ; PAIR } } };
RUN: use %default; drop all; push ('tezos', '');
CAR: pop ('tezos', ''); push tezos;
DUP: push tezos;
SIZE: pop tezos; push 5;
PUSH: push 5;
COMPARE: pop 5, 5; push 0;
LT: pop 0; push False;
IF: pop False;
NIL: push [];
PAIR: pop [], tezos; push ([], 'tezos');

value,type
"""tezos""",string


If you add another character to "tezos", you will see the contract fail 😊

One of the most common actions on strings is concatenation. Michelson provides an instruction that allows you to put two separate strings together: **`CONCAT`**. This is how it works:

In [19]:
storage string ;
parameter string ;
code {
    CAR ;
    PUSH string "Hello " ;
    CONCAT ;
    NIL operation ;
    PAIR
} ;

RUN %default "Tezos" "" ;

storage string;
parameter string;
code { CAR ; PUSH string "Hello " ; CONCAT ; NIL operation ; PAIR };
RUN: use %default; drop all; push ('Tezos', '');
CAR: pop ('Tezos', ''); push Tezos;
PUSH: push Hello ;
CONCAT: pop Hello ; pop Tezos; push Hello Tezos;
NIL: push [];
PAIR: pop [], Hello Tezos; push ([], 'Hello Tezos');

value,type
"""Hello Tezos""",string


This smart contract does a very simple thing: it takes a string as a parameter and put it together with "Hello " before saving it in the storage. When using **`CONCAT`**, just make sure you have two strings on top of the stack and that they are in the right order, the element 1 will be the first one in the new string and element 2 will be the second one.

If you want to concatenate multiple strings, you just have to add them to the stack and concatenate the **`CONCAT`** instructions!

In [32]:
storage string ;
parameter string ;
code {
    CAR ;
    PUSH string "Tezos " ;
    PUSH string "Hello " ;
    CONCAT ;
    CONCAT ;
    NIL operation ;
    PAIR
} ;

RUN %default "World" "" ;

storage string;
parameter string;
code { CAR ; PUSH string "Tezos " ; PUSH string "Hello " ; CONCAT ; CONCAT ; NIL operation ; PAIR };
RUN: use %default; drop all; push ('World', '');
CAR: pop ('World', ''); push World;
PUSH: push Tezos ;
PUSH: push Hello ;
CONCAT: pop Hello ; pop Tezos ; push Hello Tezos ;
CONCAT: pop Hello Tezos ; pop World; push Hello Tezos World;
NIL: push [];
PAIR: pop [], Hello Tezos World; push ([], 'Hello Tezos World');

value,type
"""Hello Tezos World""",string


The opposite action is **`SLICE`**: instead of putting two strings together, we are cutting one in pieces! This instruction takes three parameters: the starting point for the slice, the character length you want to slice and the string. Let's observe a few examples:

In [27]:
storage unit ;
parameter unit ;
BEGIN Unit Unit ;
DROP ;
## Pushes a string and extract the first 6 characters
PUSH @string_to_slice string "BakingBad" ;
PUSH @length nat 6 ; ## length
PUSH @offset nat 0 ; ## offset
DUMP ;

storage unit;
parameter unit;
BEGIN: use %default; drop all; push (Unit, Unit);
DROP: pop (Unit, Unit);
PUSH: push BakingBad;
PUSH: push 6;
PUSH: push 0;

value,type,name
0,nat,@offset
6,nat,@length
"""BakingBad""",string,@string_to_slice


This is how the stack looks like before the slicing begins...

In [28]:
SLICE ;

value,type
"Some ""Baking""",option string


You probably notice that the result of **`SLICE`** is wrapped in an `option` value. If the offset or the length you set go beyond the bounds of the string, the opcode returns `None`:

In [30]:
DROP ;
PUSH string "BakingBad" ;
PUSH nat 6 ; ## length
PUSH nat 6 ; ## offset
SLICE ;

DROP: pop None;
PUSH: push BakingBad;
PUSH: push 6;
PUSH: push 6;
SLICE: pop 6, 6, BakingBad; push None;

value,type
None,option string


Obviously, you are not limited to extract slices of strings from the first character, you can start at whichever position you like, keeping in mind the bounds of the string:

In [31]:
DROP ;
PUSH string "BakingBadIsAwesome" ;
PUSH nat 7 ; ## length
PUSH nat 11 ; ## offset
SLICE ;

DROP: pop None;
PUSH: push BakingBadIsAwesome;
PUSH: push 7;
PUSH: push 11;
SLICE: pop 11, 7, BakingBadIsAwesome; push ('Awesome',);

value,type
"Some ""Awesome""",option string


Slicing a string in two pieces is going to ask for a little extra work but it is totally possible. Here is how to do it:

In [36]:
DROP ;
PUSH string "BakingBadIsAwesome" ; ## Push the string to slice in two
DUP ; ## Duplicate the string
PUSH nat 9 ; ## The length of the first piece
PUSH nat 0 ; ## The offset of the first piece
SLICE ; ## Extract the first piece of the string
SWAP ; ## Get the duplicated original string on top of the stack
PUSH nat 9 ; ## The lenght of the second piece
PUSH nat 9 ; ## The offset of the second piece
SLICE ; ## Extract the second piece of the string
SWAP ; ## Put the pieces in the right order
DUMP ;

DROP_ALL: drop all;
PUSH: push BakingBadIsAwesome;
DUP: push BakingBadIsAwesome;
PUSH: push 9;
PUSH: push 0;
SLICE: pop 0, 9, BakingBadIsAwesome; push ('BakingBad',);
SWAP: pop ('BakingBad',), BakingBadIsAwesome; push ('BakingBad',); push BakingBadIsAwesome;
PUSH: push 9;
PUSH: push 9;
SLICE: pop 9, 9, BakingBadIsAwesome; push ('IsAwesome',);
SWAP: pop ('IsAwesome',), ('BakingBad',); push ('IsAwesome',); push ('BakingBad',);

value,type
"Some ""BakingBad""",option string
"Some ""IsAwesome""",option string
